In [2]:
import os
import numpy as np
import csv
import json
import re
from tqdm import tqdm
import pickle
from tqdm import tqdm
import json
def load_word():
    with open('data\\renttherunway_final_data.json\\renttherunway_final_data.json',encoding='utf-8') as file:
        words_list = []
        for data in file.readlines():
            data=json.loads(data)
            if data["rating"]==None:
                continue
            label=int(data["rating"])
            # 去除标点符号
            r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n。！，]+'
            text1=str(data["review_text"])
            text2=str(data["review_summary"])
            text1 = text1.replace('\n', '')
            text1 = text1.replace('<br /><br />', ' ')
            text1 = re.sub(r, '', text1)
            text1 = text1.split(' ')
            text1 = [text1[i].lower() for i in range(len(text1)) if text1[i] != '']
            text2 = text2.replace('\n', '')
            text2 = text2.replace('<br /><br />', ' ')
            text2 = re.sub(r, '', text2)
            text2 = text2.split(' ')
            text2 = [text2[i].lower() for i in range(len(text2)) if text2[i] != '']
            words_list.append([text1+text2, label])
        return words_list

def get_word_vector(csv_path,txt_path,vocabulary_path,word_list_path):
    vocabulary_vectors = np.load(vocabulary_path, allow_pickle=True)
    word_list = np.load(word_list_path, allow_pickle=True)
    word_list = word_list.tolist()
    true_cnt=0
    false_cnt=0
    with open(csv_path,encoding='utf-8') as file,open(txt_path,'a') as txt:
        for idx,data in tqdm(enumerate(csv.reader(file,skipinitialspace=True))):
            if idx<50000:
                if data[1]==None or data[0]=='label':
                    continue
                if true_cnt==25000:
                    if data[0]=='1':
                        continue
                if false_cnt==25000:
                    if data[0]=='0':
                        continue
                label=int(data[0])
                if label=='0':
                    false_cnt+=1
                else:
                    true_cnt+=1
                # 去除标点符号
                r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n。！，]+'
                text1=str(data[1])
                text2=str(data[9])
                text1 = text1.replace('\n', '')
                text1 = text1.replace('<br /><br />', ' ')
                text1 = re.sub(r, '', text1)
                text1 = text1.split(' ')
                text1 = [text1[i].lower() for i in range(len(text1)) if text1[i] != '']
                text2 = text2.replace('\n', '')
                text2 = text2.replace('<br /><br />', ' ')
                text2 = re.sub(r, '', text2)
                text2 = text2.split(' ')
                text2 = [text2[i].lower() for i in range(len(text2)) if text2[i] != '']
                # print(i)
                sentence = text1+text2
                temp = []
                text1_data=[]
                text2_data=[]
                index = 0
                for j in range(len(sentence)):
                    try:
                        index = word_list.index(sentence[j])
                    except ValueError:  # 没找到
                        index = 399999
                    finally:
                        temp.append(list(vocabulary_vectors[index]))  # temp表示一个单词在词典中的序号
                '''
                if len(temp) < 100:
                    for k in range(len(temp), 100):  # 不足补0
                        temp.append([0]*100)
                else:
                    temp = temp[0:50]  # 只保留250个
                '''
                for j in range(len(text1)):
                    try:
                        index = word_list.index(text1[j])
                    except ValueError:  # 没找到
                        index = 399999
                    finally:
                        text1_data.append(list(vocabulary_vectors[index]))  # temp表示一个单词在词典中的序号
                for j in range(len(text2)):
                    try:
                        index = word_list.index(text2[j])
                    except ValueError:  # 没找到
                        index = 399999
                    finally:
                        text2_data.append(list(vocabulary_vectors[index]))  # temp表示一个单词在词典中的序号
                data_dict={"data":temp,"label":label,"text1":text1_data,"text2":text2_data}
                json_data=json.dumps(data_dict)
                txt.writelines(json_data+'\n')



In [3]:
get_word_vector('/content/drive/Shareddrives/CSE256/data/train-balanced-sarcasm.csv','/content/drive/Shareddrives/CSE256/data/train_data_100d.txt','/content/drive/Shareddrives/CSE256/data/vocabulary_vectors_100d.npy','/content/drive/Shareddrives/CSE256/data/word_list_100d.npy')

1010827it [30:15, 556.76it/s]  


In [ ]:
with open('/content/drive/Shareddrives/CSE256/data/train_data.pkl','rb') as file:
    data=pickle.load(file)
    print(data)

FileNotFoundError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
